In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


Vamos testar alguns modelos diferentes de classificação, a fim de explorarmos algumas possibilidades. Dado que já fizemos uma análise exploratória, sabemos quais variáveis podemos começar incluindo no modelo para nos ajudar a obter um bom preditor para nossa variável de interesse. Nesse momento iremos testar 3 modelos diferentes:

- Regressão Logística 
- Random Forest
- XgBoost

Todos os 3 devem ser suficientes para obtermos um bom modelo com o conjunto de dados de interesse, o que irá diferenciar é a performance de cada um deles, veremos...

In [33]:
# Importando Dataset:
df_heart = pd.read_csv(r"C:\Users\lucaa\Desktop\Pastas Gerais\Pós Tech Data Analytics\Tech Challenge\TechChallenge - Fase 3\techc_fase3_ml\data\raw\dataset_heart.csv", sep = ",")
df_heart.head(3)

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1


In [34]:
# Ajuste Inicial - separando conjuntos:
X = df_heart.drop(columns = ["DEATH_EVENT", "time"]) # features
y = df_heart["DEATH_EVENT"] # target

# Splitando dataset para treinamento:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 23)

# Totais de cada set:
print(f'''
Total para treino: {len(y_train)}
Total para teste: {len(y_test)}
''')



Total para treino: 209
Total para teste: 90



In [48]:
# Features:
X_train.columns

Index(['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes',
       'ejection_fraction', 'high_blood_pressure', 'platelets',
       'serum_creatinine', 'serum_sodium', 'sex', 'smoking'],
      dtype='object')

In [35]:
# Definindo os modelos
log_reg = LogisticRegression(random_state=42)
random_forest = RandomForestClassifier(random_state=42)
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

In [36]:
# Treinando os modelos
log_reg.fit(X_train, y_train)
random_forest.fit(X_train, y_train) 
xgb.fit(X_train, y_train)

c:\Users\lucaa\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [07:55:46] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [37]:
# Realizando predições
y_pred_log_reg = log_reg.predict(X_test)
y_pred_rf = random_forest.predict(X_test)
y_pred_xgb = xgb.predict(X_test)

In [38]:
def evaluate_model(model_name, y_true, y_pred):
    print(f"Performance {model_name}:")
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precision: {precision_score(y_true, y_pred):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred):.4f}")
    print(f"F1 Score: {f1_score(y_true, y_pred):.4f}")
    print("-" * 30)

In [39]:
# Evaluate each model
evaluate_model("Logistic Regression", y_test, y_pred_log_reg)
evaluate_model("Random Forest", y_test, y_pred_rf)
evaluate_model("XGBoost", y_test, y_pred_xgb)

Performance Logistic Regression:
Accuracy: 0.7222
Precision: 0.7692
Recall: 0.3125
F1 Score: 0.4444
------------------------------
Performance Random Forest:
Accuracy: 0.6778
Precision: 0.5600
Recall: 0.4375
F1 Score: 0.4912
------------------------------
Performance XGBoost:
Accuracy: 0.7000
Precision: 0.5926
Recall: 0.5000
F1 Score: 0.5424
------------------------------


Aqui vemos como é simples ajustar os 3 modelos escolhidos e também como a performance dos 3 é similar, porém o trabalho de ajuste não acaba apenas aqui, agora podemos focar em outras técnicas para melhorar a performance de cada um deles e eventualmente escolher o mais interessante. De cara podemos ver que a regressão logística foi a que menos performou como um todo, mas isso é esperado dado que é um algoritmo supervisionado menos robusto e é mais próximo de um modelo paramétrico do que um algoritmo de Machine Learning. 

Além disso, antes do tuning espera-se também que a Random Forest e o XGBoost tenham performances semelhantes, dado que ambos são algoritmos baseados em árvores, agora iremos seguir com os dois a fim de verificarmos qual nos trará o melhor resultado ao final.

#### Tuning Random Forest

In [40]:
# Especificando um grid de parametros para Tuning:
param_grid_rf = {
    'n_estimators': [100, 200, 300],   # Número de árvores
    'max_depth': [10, 20, 30],         # Depth das árvores
    'min_samples_split': [2, 5, 10],   
    'min_samples_leaf': [1, 2, 4]
}

In [41]:
# Utilziando Grid Search para Tuning com Cross Validation:
grid_search_rf = GridSearchCV(
    estimator=random_forest, 
    param_grid=param_grid_rf, 
    cv=10, 
    scoring='accuracy', 
    n_jobs=-1, 
    verbose=1
)


In [42]:
# Fit the model with grid search
grid_search_rf.fit(X_train, y_train)

Fitting 10 folds for each of 81 candidates, totalling 810 fits


GridSearchCV(cv=10, estimator=RandomForestClassifier(random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': [10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='accuracy', verbose=1)

In [43]:
# Melhores parametros para RF:
best_params_rf = grid_search_rf.best_params_
print(f"Melhores Parametros: {best_params_rf}")

# Melhor Modelo Ajustado:
best_rf = grid_search_rf.best_estimator_

# Evaluate the model on the test set
accuracy_rf = best_rf.score(X_test, y_test)
print(f"Test set accuracy: {accuracy_rf:.8f}")

Melhores Parametros: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 300}
Test set accuracy: 0.72222222


#### Tuning XgBoost

In [44]:
param_grid_xg = {
    'n_estimators': [100, 200, 300],  # Number of boosting rounds (trees)
    'max_depth': [3, 5, 7],           # Maximum depth of a tree
    'learning_rate': [0.01, 0.1, 0.2],# Step size shrinkage used in updates
    'subsample': [0.8, 1.0],          # Fraction of samples used per tree
    'colsample_bytree': [0.8, 1.0],   # Fraction of features used per tree
    'gamma': [0, 0.1, 0.2],           # Minimum loss reduction required for a split
}

In [45]:
# Utilziando Grid Search para Tuning:
grid_search_xg = GridSearchCV(
    estimator=xgb, 
    param_grid=param_grid_xg, 
    cv=10, 
    scoring='accuracy', 
    n_jobs=-1, 
    verbose=1
)


In [46]:
# Fit the model with grid search
grid_search_xg.fit(X_train, y_train)

Fitting 10 folds for each of 324 candidates, totalling 3240 fits


c:\Users\lucaa\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\externals\loky\process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
c:\Users\lucaa\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [07:57:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


GridSearchCV(cv=10,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='logloss', feature_types=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.8, 1.0], 'gamma': [0, 0.1, 0.2],
                         'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [100, 200, 300],
                         'subsample': [0.8, 1.0]},
             scoring='accuracy', verbose=1)

In [47]:
# Melhores parametros para RF:
best_params_xg = grid_search_xg.best_params_
print(f"Melhores Parametros: {best_params_xg}")

# Melhor Modelo Ajustado:
best_xg = grid_search_xg.best_estimator_

# Evaluate the model on the test set
accuracy_xg = best_xg.score(X_test, y_test)
print(f"Test set accuracy: {accuracy_xg:.8f}")

Melhores Parametros: {'colsample_bytree': 1.0, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 200, 'subsample': 1.0}
Test set accuracy: 0.67777778


#### Modelo Final

In [49]:
# Comparando resultados pela Accuray:
print(f'''Acurácias:
XGBoost: {accuracy_xg}
Random Forest: {accuracy_xg}
''')

Acurácias:
XGBoost: 0.6777777777777778
Random Forest: 0.6777777777777778



Nos testes em geral, tivemos desempenhos muito similares entre os 2 algoritmos, e como nem sempre o mais complexo é melhor iremos seguir com a Random Forest para implementação final e utilização na aplicação. Lembrando que um modelo nunca acaba nos ajustes de treinamento, agora que escolhemos nossos parâmetros finais, precisamos re ajustar o modelo com todas as informações disponíveis para ai sim realizarmos o deploy em produção.

In [61]:
best_params_rf

{'max_depth': 20,
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 300}

In [62]:
# Ajustando o modelo final:
random_forest_final = RandomForestClassifier(
    n_estimators = best_params_rf["n_estimators"],
    max_depth = best_params_rf["max_depth"],
    min_samples_split= best_params_rf["min_samples_split"],
    min_samples_leaf= best_params_rf["min_samples_leaf"],
    random_state=42
)

In [63]:
random_forest_final.fit(X, y)

RandomForestClassifier(max_depth=20, min_samples_split=5, n_estimators=300,
                       random_state=42)

In [64]:
# Dumping to Pickle:
import pickle

In [65]:
with open(r'C:\Users\lucaa\Desktop\Pastas Gerais\Pós Tech Data Analytics\Tech Challenge\TechChallenge - Fase 3\techc_fase3_ml\models\random_forest_model.pkl', 'wb') as file:
    pickle.dump(random_forest_final, file)
print("Modelo Exportado em Pickle para 'random_forest_model.pkl'")

Modelo Exportado em Pickle para 'random_forest_model.pkl'


In [72]:
import numpy as np

In [77]:
with open(r'C:\Users\lucaa\Desktop\Pastas Gerais\Pós Tech Data Analytics\Tech Challenge\TechChallenge - Fase 3\techc_fase3_ml\models\random_forest_model.pkl', 'rb') as file:
    loaded_rf_model = pickle.load(file)

# Assuming X_new is your new data for which you want to make predictions
# Example: X_new = [[feature1_value, feature2_value, ..., featureN_value]]

[55.0, 0.0, 7861.0, 0.0, 38.0, 0.0, 263358.03, 1.1, 136.0, 1.0, 0.0]

In [91]:
predictions = loaded_rf_model.predict_proba([X.iloc[1].to_list()])

c:\Users\lucaa\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [92]:
predictions

array([[0.2880582, 0.7119418]])

In [93]:
y.iloc[1]

1